In [1]:
import pandas as pd
import numpy as np
import json

In [51]:
train = pd.read_csv('data/train.csv', sep=';', delimiter=',', 
                    dtype={'fullVisitorId' : str, 'visitId' : str}).sample(frac=0.1).reset_index(drop=True)

In [153]:
train.info()
train.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90365 entries, 0 to 90364
Data columns (total 33 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   channelGrouping                 90365 non-null  int8          
 1   date                            90365 non-null  datetime64[ns]
 2   fullVisitorId                   90365 non-null  object        
 3   sessionId                       90365 non-null  object        
 4   visitId                         90365 non-null  object        
 5   visitNumber                     90365 non-null  int64         
 6   visitStartTime                  90365 non-null  datetime64[ns]
 7   browser                         90365 non-null  object        
 8   operatingSystem                 90365 non-null  int8          
 9   isMobile                        90365 non-null  bool          
 10  deviceCategory                  90365 non-null  object        
 11  co

,channelGrouping,visitNumber,operatingSystem,visits,hits,pageviews,newVisits,bounces,transactionRevenue
count,90365.000000,90365.000000,90365.000000,90365.0,90365.000000,90365.000000,90365.000000,90365.000000,9.036500e+04
mean,4.639838,2.285841,8.360062,1.0,4.585227,3.851392,0.777890,0.499452,1.644697e+06
std,1.780916,9.584053,3.996299,0.0,9.643410,7.143047,0.415667,0.500002,4.244214e+07
min,0.000000,1.000000,0.000000,1.0,1.000000,1.000000,0.000000,0.000000,0.000000e+00
25%,4.000000,1.000000,7.000000,1.0,1.000000,1.000000,1.000000,0.000000,0.000000e+00
50%,4.000000,1.000000,11.000000,1.0,2.000000,1.000000,1.000000,0.000000,0.000000e+00
75%,7.000000,1.000000,11.000000,1.0,4.000000,4.000000,1.000000,1.000000,0.000000e+00
max,7.000000,383.000000,14.000000,1.0,500.000000,429.000000,1.000000,1.000000,8.677830e+09


In [129]:
train.head()

,channelGrouping,date,fullVisitorId,sessionId,visitId,visitNumber,visitStartTime,browser,operatingSystem,isMobile,...,medium,isTrueDirect,referralPath,keyword,adwordsClickInfo.page,adwordsClickInfo.slot,adwordsClickInfo.gclId,adwordsClickInfo.adNetworkType,adwordsClickInfo.isVideoAd,adContent
0,2,2017-07-05,9197671230459911377,9197671230459911377_1499240705,1499240705,2,2017-07-05 07:45:05,Chrome,Android,True,...,(none),True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2017-07-11,9374539600061108931,9374539600061108931_1499841817,1499841817,1,2017-07-12 06:43:37,Chrome,Macintosh,False,...,(none),True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7,2016-10-21,4983350835916033613,4983350835916033613_1477091760,1477091760,1,2016-10-21 23:16:00,Chrome,Windows,False,...,referral,False,/yt/about/es/,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,2016-12-03,3738915998689863059,3738915998689863059_1480779478,1480779478,3,2016-12-03 15:37:58,Chrome,Android,True,...,cpc,True,NaN,6qEhsCssdK0z36ri,1,Top,Cj0KEQiA6_TBBRDInaPjhcelt5oBEiQApPeTFxk8JPQydz...,Google Search,False,NaN
4,4,2017-02-10,9933232621394732713,9933232621394732713_1486717881,1486717881,1,2017-02-10 09:11:21,Chrome,Macintosh,False,...,organic,False,NaN,(not provided),NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
train.isTrueDirect = train.isTrueDirect.fillna(False)
train.hits = train.hits.astype(int)
train.visits = train.visits.astype(int)
train.pageviews = train.pageviews.fillna(train.pageviews.astype(float).mean())
train.pageviews = train.pageviews.astype(int)
train.newVisits = train.newVisits.fillna(0)
train.newVisits = train.newVisits.astype(int)
train.bounces = train.bounces.fillna(0).astype(int)
train.transactionRevenue = train.transactionRevenue.fillna(0).astype(float)

In [125]:
train.channelGrouping.value_counts()

Organic Search    37709
Social            22896
Direct            14422
Referral          10447
Paid Search        2626
Affiliates         1616
Display             633
(Other)              16
Name: channelGrouping, dtype: int64

In [54]:
train.date = pd.to_datetime(train.date, format='%Y%m%d')
train.visitStartTime = pd.to_datetime(train.visitStartTime, unit='s')
train = pd.merge(left=train, right=pd.json_normalize(train.device.apply(json.loads)), 
                 left_index=True, right_index=True).drop('device', axis=1)
train = pd.merge(left=train, right=pd.json_normalize(train.geoNetwork.apply(json.loads)), 
                 left_index=True, right_index=True).drop('geoNetwork', axis=1)
train = pd.merge(left=train, right=pd.json_normalize(train.totals.apply(json.loads)), 
                 left_index=True, right_index=True).drop('totals', axis=1)
train = pd.merge(left=train, right=pd.json_normalize(train.trafficSource.apply(json.loads)), 
                 left_index=True, right_index=True).drop('trafficSource', axis=1)

In [62]:
for c in train:
    if train[c][0] == 'not available in demo dataset':
        train = train.drop(c, axis=1)

In [63]:
train = train.drop('socialEngagementType', axis=1)

In [24]:
!pip install mlxtend

In [158]:
df = pd.merge(left=train.fullVisitorId, right=train.select_dtypes(include=[float, np.int8, int, bool]), left_index=True, right_index=True)
df = df.groupby('fullVisitorId').sum().reset_index()

In [159]:
X = df.drop(['fullVisitorId', 'transactionRevenue'], axis=1)
y = np.log(df.transactionRevenue + 1)

In [160]:
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression

selector = SequentialFeatureSelector(LinearRegression(), verbose=2, k_features=5, forward=False, n_jobs=-1)
selector.fit(X, y)
selector.subsets_

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   9 | elapsed:    0.6s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:    0.8s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:    0.8s finished

[2020-05-05 18:02:37] Features: 8/5 -- score: 0.18096298059943797[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   8 | elapsed:    0.5s remaining:    0.8s
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:    0.5s finished

[2020-05-05 18:02:37] Features: 7/5 -- score: 0.18066640613083912[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   7 | elapsed:    0.3s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done   7 out of   7 | elapsed:    0.4s finished

[2020-05-05 18:02:38]

{9: {'feature_idx': (0, 1, 2, 3, 4, 5, 6, 7, 8),
  'cv_scores': array([0.18874394, 0.19469652, 0.19157034, 0.19040062, 0.11875888]),
  'avg_score': 0.17683405975970223,
  'feature_names': ('channelGrouping',
   'operatingSystem',
   'isMobile',
   'visits',
   'hits',
   'pageviews',
   'newVisits',
   'bounces',
   'isTrueDirect')},
 8: {'feature_idx': (0, 1, 2, 3, 5, 6, 7, 8),
  'cv_scores': array([0.18364751, 0.19335868, 0.18581129, 0.18435182, 0.1576456 ]),
  'avg_score': 0.18096298059943797,
  'feature_names': ('channelGrouping',
   'operatingSystem',
   'isMobile',
   'visits',
   'pageviews',
   'newVisits',
   'bounces',
   'isTrueDirect')},
 7: {'feature_idx': (0, 1, 2, 3, 5, 6, 7),
  'cv_scores': array([0.18369737, 0.1930939 , 0.1857202 , 0.18370788, 0.15711267]),
  'avg_score': 0.18066640613083912,
  'feature_names': ('channelGrouping',
   'operatingSystem',
   'isMobile',
   'visits',
   'pageviews',
   'newVisits',
   'bounces')},
 6: {'feature_idx': (1, 2, 3, 5, 6, 7),
  

In [151]:
train.channelGrouping = train.channelGrouping.astype('category').cat.codes

In [152]:
train.operatingSystem = train.operatingSystem.astype('category').cat.codes

In [161]:
X

,channelGrouping,operatingSystem,isMobile,visits,hits,pageviews,newVisits,bounces,isTrueDirect
0,7.0,11.0,0.0,1,2,2,1,0,0.0
1,6.0,7.0,0.0,1,8,8,0,0,1.0
2,6.0,3.0,0.0,1,2,2,1,0,0.0
3,7.0,7.0,0.0,1,1,1,1,1,0.0
4,4.0,1.0,1.0,1,1,1,0,1,1.0
...,...,...,...,...,...,...,...,...,...
85833,4.0,11.0,0.0,1,1,1,1,1,0.0
85834,7.0,11.0,0.0,1,1,1,1,1,0.0
85835,7.0,7.0,0.0,1,1,1,1,1,0.0
85836,7.0,11.0,0.0,1,2,1,1,1,0.0
